### Word Group In Youtube

In [83]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [84]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [85]:
# language pair
lang_folder = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 20000  # native word end index 28,200,1000,5000,10000, 20000, 40000

# word ratio
if word_end == 28 or word_end == 200:
    unique_word_ratio = 70
    include_wordgroup_ratio = 70
elif word_end == 1000 or word_end == 5000:
    unique_word_ratio = 80
    include_wordgroup_ratio = 80
elif word_end == 10000:
    unique_word_ratio = 85
    include_wordgroup_ratio = 85
else:
    unique_word_ratio = 90
    include_wordgroup_ratio = 90

# word usage
if word_end == 28 or word_end == 200 or word_end == 1000 or word_end == 5000 or word_end == 10000:
    word_usage_min = 10
    word_usage_max = 100
elif word_end == 20000:
    word_usage_min = 7
    word_usage_max = 100
else:
    word_usage_min = 5
    word_usage_max = 100

# use for manuel set
#unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
#include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
#word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
#word_usage_max = 100


# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"word_end:{word_end}\nfile_ext:{file_ext}\nunique_word_ratio:{unique_word_ratio}\n\
include_wordgroup_ratio:{include_wordgroup_ratio}\nword_usage_min:{word_usage_min}\nword_usage_max:{word_usage_max}")

word_end:200
file_ext:6
unique_word_ratio:70
include_wordgroup_ratio:70
word_usage_min:10
word_usage_max:100


In [86]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [87]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [88]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [89]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [90]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [91]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [92]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
552149,fruitcocktail,6
552150,andthesunlightshining,6
552151,upravo,6
552152,yagawa,6


In [93]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [94]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
195,through,1387237
196,stay,1385820
197,every,1384681
198,coming,1376868


In [95]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [96]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [97]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

199

#### Youtube Sentence Analysis

In [98]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:29.416,00:00:33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,00:00:33.810,00:00:36.876,shivering cold and snow,tPffr9WPl9Y
2,00:00:37.150,00:00:39.000,what else do you wish for in this festive season,tPffr9WPl9Y
3,00:00:39.257,00:00:43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
4,00:00:43.796,00:00:50.094,so today we will explore different places in e...,tPffr9WPl9Y
...,...,...,...,...
5742563,00:02:59.400,00:03:03.030,wounded and,78VHQeebbWM
5742564,00:03:03.030,00:03:10.590,general staff of the armed forces of ukraine o...,78VHQeebbWM
5742565,00:03:10.590,00:03:13.159,facebook on december,78VHQeebbWM
5742566,00:03:13.159,00:03:16.769,you can express your thoughts in the comments ...,78VHQeebbWM


In [99]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [100]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y
...,...,...,...,...,...
5742563,5742563,179.400,183.030,wounded and,78VHQeebbWM
5742564,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM
5742565,5742565,190.590,193.159,facebook on december,78VHQeebbWM
5742566,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM


In [101]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [102]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [103]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,37.5,"jingle, bell","way, all, the"
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,25.0,"cold, snow, shivering",and
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,60.0,"festive, else, wish, season","for, this, in, you, what, do"
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,35.7,"edinburgh, herself, christmas, colours, decora...","has, and, the, with, of"
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,47.1,"edinburgh, places, celebrated, explore, differ...","so, is, in, here, how, and, we, will"
...,...,...,...,...,...,...,...,...
5742563,5742563,179.400,183.030,wounded and,78VHQeebbWM,50.0,wounded,and
5742564,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,35.7,"staff, operational, shared, forces, general, a...","has, been, on, the, of"
5742565,5742565,190.590,193.159,facebook on december,78VHQeebbWM,33.3,"december, facebook",on
5742566,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,55.5,"section, express, thoughts, comments","can, your, in, you, the"


In [104]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,37.5,"jingle, bell","way, all, the"
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,25.0,"cold, snow, shivering",and
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,60.0,"festive, else, wish, season","for, this, in, you, what, do"
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,35.7,"edinburgh, herself, christmas, colours, decora...","has, and, the, with, of"
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,47.1,"edinburgh, places, celebrated, explore, differ...","so, is, in, here, how, and, we, will"
...,...,...,...,...,...,...,...,...
5742563,5742563,179.400,183.030,wounded and,78VHQeebbWM,50.0,wounded,and
5742564,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,35.7,"staff, operational, shared, forces, general, a...","has, been, on, the, of"
5742565,5742565,190.590,193.159,facebook on december,78VHQeebbWM,33.3,"december, facebook",on
5742566,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,55.5,"section, express, thoughts, comments","can, your, in, you, the"


In [105]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,37.5,"jingle, bell","way, all, the"
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,25.0,"cold, snow, shivering",and
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,60.0,"festive, else, wish, season","for, this, in, you, what, do"
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,35.7,"edinburgh, herself, christmas, colours, decora...","has, and, the, with, of"
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,47.1,"edinburgh, places, celebrated, explore, differ...","so, is, in, here, how, and, we, will"
...,...,...,...,...,...,...,...,...
5410983,5742563,179.400,183.030,wounded and,78VHQeebbWM,50.0,wounded,and
5410984,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,35.7,"staff, operational, shared, forces, general, a...","has, been, on, the, of"
5410985,5742565,190.590,193.159,facebook on december,78VHQeebbWM,33.3,"december, facebook",on
5410986,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,55.5,"section, express, thoughts, comments","can, your, in, you, the"


In [106]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

<ipython-input-106-ca15520b3270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
8,8,66.866,70.225,so there is no point of sitting in home in thi...,tPffr9WPl9Y,64.3,"year, point, sitting","there, so, time, this, is, no, in, home, of",in home in this time of
36,36,290.380,294.219,decorative items are not the only thing in the...,tPffr9WPl9Y,60.0,"decorative, fair, items","only, in, not, the, are, thing",are not the only thing in the
55,63,436.499,440.769,i am now in a maze of christmas trees,tPffr9WPl9Y,66.7,"trees, maze, christmas","now, in, i, a, am, of",i am now in a
57,65,443.400,450.297,we have been provided with a sheet and we have...,tPffr9WPl9Y,57.9,"hidden, sheet, letters, provided, maze, seven","have, been, in, find, out, we, and, the, with,...",and we have to find out
58,66,450.959,456.239,after finding those letters we need to rearran...,tPffr9WPl9Y,56.2,"word, christmas, letters, rearrange, finding, ...","those, need, them, find, out, after, we, a, to",them to find out a
...,...,...,...,...,...,...,...,...,...
5410943,5742520,0.000,3.420,uh this is gon na be a little corny sounding b...,K70K3qvv4Bo,75.0,"sounding, honestly, corny","be, uh, this, is, gon, na, but, little, a",uh this is gon na be a little
5410944,5742521,4.200,6.240,on stage i mean this is all,K70K3qvv4Bo,85.7,stage,"this, is, i, all, on, mean",i mean this is all
5410946,5742523,10.680,14.040,anywhere i can get up with a crowd of people a...,K70K3qvv4Bo,73.3,"play, crowd, music, anywhere","can, people, for, my, get, i, and, up, with, a...",i can get up with a
5410947,5742524,14.040,17.040,them and hear them singing it back i mean its ...,K70K3qvv4Bo,68.7,"hear, its, singing","there, them, back, is, i, it, really, all, and...",really all there is to it


In [107]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,in home in this time of,68.601483,70.001067,so there is no point of sitting in home in thi...,tPffr9WPl9Y
1,are not the only thing in the,291.584392,293.917902,decorative items are not the only thing in the...,tPffr9WPl9Y
2,i am now in a,436.499000,438.114676,i am now in a maze of christmas trees,tPffr9WPl9Y
3,and we have to find out,445.976901,447.871681,we have been provided with a sheet and we have...,tPffr9WPl9Y
4,them to find out a,453.775000,454.948333,after finding those letters we need to rearran...,tPffr9WPl9Y
...,...,...,...,...,...
1053639,uh this is gon na be a little,0.000000,1.800000,uh this is gon na be a little corny sounding b...,K70K3qvv4Bo
1053640,i mean this is all,4.804444,6.240000,on stage i mean this is all,K70K3qvv4Bo
1053641,i can get up with a,11.087273,12.156364,anywhere i can get up with a crowd of people a...,K70K3qvv4Bo
1053642,really all there is to it,15.941408,17.040000,them and hear them singing it back i mean its ...,K70K3qvv4Bo


In [108]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,in home in this time of,68.301483,70.301067,so there is no point of sitting in home in thi...,tPffr9WPl9Y
1,are not the only thing in the,291.284392,294.217902,decorative items are not the only thing in the...,tPffr9WPl9Y
2,i am now in a,436.199000,438.414676,i am now in a maze of christmas trees,tPffr9WPl9Y
3,and we have to find out,445.676901,448.171681,we have been provided with a sheet and we have...,tPffr9WPl9Y
4,them to find out a,453.475000,455.248333,after finding those letters we need to rearran...,tPffr9WPl9Y
...,...,...,...,...,...
1053639,uh this is gon na be a little,-0.300000,2.100000,uh this is gon na be a little corny sounding b...,K70K3qvv4Bo
1053640,i mean this is all,4.504444,6.540000,on stage i mean this is all,K70K3qvv4Bo
1053641,i can get up with a,10.787273,12.456364,anywhere i can get up with a crowd of people a...,K70K3qvv4Bo
1053642,really all there is to it,15.641408,17.340000,them and hear them singing it back i mean its ...,K70K3qvv4Bo


In [109]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,in home in this time of,68,70,so there is no point of sitting in home in thi...,tPffr9WPl9Y
1,are not the only thing in the,291,294,decorative items are not the only thing in the...,tPffr9WPl9Y
2,i am now in a,436,438,i am now in a maze of christmas trees,tPffr9WPl9Y
3,and we have to find out,446,448,we have been provided with a sheet and we have...,tPffr9WPl9Y
4,them to find out a,453,455,after finding those letters we need to rearran...,tPffr9WPl9Y
...,...,...,...,...,...
1053639,uh this is gon na be a little,0,2,uh this is gon na be a little corny sounding b...,K70K3qvv4Bo
1053640,i mean this is all,5,7,on stage i mean this is all,K70K3qvv4Bo
1053641,i can get up with a,11,12,anywhere i can get up with a crowd of people a...,K70K3qvv4Bo
1053642,really all there is to it,16,17,them and hear them singing it back i mean its ...,K70K3qvv4Bo


In [110]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,you know i could stay here all day with you bu...,2845,2854,james moore you know i could stay here all day...,M_IRnJZ2xQ0
1,on but you want to find that out but you got i...,2017,2023,velera wilson if you dont have to stay there f...,UErS7dSY59M
2,so do you think life will work in the right wa...,560,567,so do you think life will work in the right wa...,IXhNNNeYXnA
3,what do you think about this i mean yeah to me...,538,546,what do you think about this i mean yeah to me...,_aBzOI9DWb4
4,things that are out there you can do nothing a...,937,943,things that are out there you can do nothing a...,ldlflaQVAAE
...,...,...,...,...,...
1053639,or or a a a,238,239,a food which isserved on a plate or or a a a s...,3gFd5H68kpA
1053640,i i do i am,4074,4076,branca dunno spice i i do i am working of cour...,WSrGL9E2wD4
1053641,a is a is a,3057,3059,a is a is a matrix that switches from one,rBE7izu2Zc0
1053642,i do i am a,36,37,do you have a job there brandon i do i am a do...,hj5DfJMUjT4


In [111]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,you know i could stay here all day with you bu...,2845,2854,james moore you know i could stay here all day...,M_IRnJZ2xQ0,75
1,on but you want to find that out but you got i...,2017,2023,velera wilson if you dont have to stay there f...,UErS7dSY59M,75
2,so do you think life will work in the right wa...,560,567,so do you think life will work in the right wa...,IXhNNNeYXnA,91
3,what do you think about this i mean yeah to me...,538,546,what do you think about this i mean yeah to me...,_aBzOI9DWb4,80
4,things that are out there you can do nothing a...,937,943,things that are out there you can do nothing a...,ldlflaQVAAE,90
...,...,...,...,...,...,...
1053639,or or a a a,238,239,a food which isserved on a plate or or a a a s...,3gFd5H68kpA,40
1053640,i i do i am,4074,4076,branca dunno spice i i do i am working of cour...,WSrGL9E2wD4,60
1053641,a is a is a,3057,3059,a is a is a matrix that switches from one,rBE7izu2Zc0,40
1053642,i do i am a,36,37,do you have a job there brandon i do i am a do...,hj5DfJMUjT4,80


In [112]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-112-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-112-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,you know i could stay here all day with you bu...,2845,2854,james moore you know i could stay here all day...,M_IRnJZ2xQ0
1,on but you want to find that out but you got i...,2017,2023,velera wilson if you dont have to stay there f...,UErS7dSY59M
2,so do you think life will work in the right wa...,560,567,so do you think life will work in the right wa...,IXhNNNeYXnA
3,what do you think about this i mean yeah to me...,538,546,what do you think about this i mean yeah to me...,_aBzOI9DWb4
4,things that are out there you can do nothing a...,937,943,things that are out there you can do nothing a...,ldlflaQVAAE
...,...,...,...,...,...
1020178,now i is a i,7852,7854,injection of a vaccine now i is a i easy any n...,GU7Y5jVQgvA
1020179,i d i had a,550,552,mouth really for a few months i mean i ha i d ...,EDP7dpubSPE
1020180,to do a d d,2224,2226,polo lounge and talk to film producers because...,C5ghwxmgMHQ
1020181,i do i am a,36,37,do you have a job there brandon i do i am a do...,hj5DfJMUjT4


In [113]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-113-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,you know i could stay here all day with you bu...,2845,2854,james moore you know i could stay here all day...,M_IRnJZ2xQ0,66
1,on but you want to find that out but you got i...,2017,2023,velera wilson if you dont have to stay there f...,UErS7dSY59M,38
2,so do you think life will work in the right wa...,560,567,so do you think life will work in the right wa...,IXhNNNeYXnA,89
3,what do you think about this i mean yeah to me...,538,546,what do you think about this i mean yeah to me...,_aBzOI9DWb4,100
4,things that are out there you can do nothing a...,937,943,things that are out there you can do nothing a...,ldlflaQVAAE,100
...,...,...,...,...,...,...
1020178,now i is a i,7852,7854,injection of a vaccine now i is a i easy any n...,GU7Y5jVQgvA,24
1020179,i d i had a,550,552,mouth really for a few months i mean i ha i d ...,EDP7dpubSPE,11
1020180,to do a d d,2224,2226,polo lounge and talk to film producers because...,C5ghwxmgMHQ,11
1020181,i do i am a,36,37,do you have a job there brandon i do i am a do...,hj5DfJMUjT4,18


In [114]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-114-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,so do you think life will work in the right wa...,560,567,so do you think life will work in the right wa...,IXhNNNeYXnA
1,what do you think about this i mean yeah to me...,538,546,what do you think about this i mean yeah to me...,_aBzOI9DWb4
2,things that are out there you can do nothing a...,937,943,things that are out there you can do nothing a...,ldlflaQVAAE
3,the way is where this is why i really want to ...,2610,2615,the way is where this is why i really want to ...,MXGmKbjEIlw
4,say oh yeah yes she can and she would like to ...,1637,1644,say oh yeah yes she can and she would like to ...,dYa5scWadpc
...,...,...,...,...,...
146318,i had it as a,3457,3459,i had it as a kid,W5h9l0jGmqM
146319,if i do it in,2251,2254,if i do it in eml,U89ZcNXZDHQ
146320,is it i or me,10,12,is it i or me,YEuDThr74dA
146321,do it at a a,9161,9169,do it at a a,CmJE6zOE4cY


In [115]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,there were a lot of people who said they were,zyHhANpL2xk
1,even know her last name,zxcNJO9nlsw
2,so much had not been going well or not been go...,zxK4hh9EhVs
3,or maybe even something more,zv7CUChuI_c
4,or not anything was going well,ztMBrL21bP4
...,...,...
1129,there was some money for some but just,0F4N_GXNBg8
1130,more money there was some money for some mr,0F4N_GXNBg8
1131,was here he would say something like,0BsyLPv4EQc
1132,this again or i want to talk about this some m...,07FIRtgrPnI


In [116]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,so do you think life will work in the right wa...,560,567,so do you think life will work in the right wa...,IXhNNNeYXnA
1,what do you think about this i mean yeah to me...,538,546,what do you think about this i mean yeah to me...,_aBzOI9DWb4
2,things that are out there you can do nothing a...,937,943,things that are out there you can do nothing a...,ldlflaQVAAE
3,the way is where this is why i really want to ...,2610,2615,the way is where this is why i really want to ...,MXGmKbjEIlw
4,say oh yeah yes she can and she would like to ...,1637,1644,say oh yeah yes she can and she would like to ...,dYa5scWadpc
...,...,...,...,...,...
1116,oh man oh my god,577,579,oh man oh my god,gi6SYQyVIf8
1117,oh no oh my god,106,108,oh no oh my god,Vnx9ZJS5Oes
1118,oh no oh my god,721,724,oh no oh my god ohhhh,Poe76zLndr8
1119,oh d oh how did,91,95,oh d oh how did,mVhs_hOfx2I


In [117]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,so do you think life will work in the right wa...,560,567,so do you think life will work in the right wa...,IXhNNNeYXnA,https://www.youtube.com/watch?v=IXhNNNeYXnA&t=...
1,what do you think about this i mean yeah to me...,538,546,what do you think about this i mean yeah to me...,_aBzOI9DWb4,https://www.youtube.com/watch?v=_aBzOI9DWb4&t=...
2,things that are out there you can do nothing a...,937,943,things that are out there you can do nothing a...,ldlflaQVAAE,https://www.youtube.com/watch?v=ldlflaQVAAE&t=...
3,the way is where this is why i really want to ...,2610,2615,the way is where this is why i really want to ...,MXGmKbjEIlw,https://www.youtube.com/watch?v=MXGmKbjEIlw&t=...
4,say oh yeah yes she can and she would like to ...,1637,1644,say oh yeah yes she can and she would like to ...,dYa5scWadpc,https://www.youtube.com/watch?v=dYa5scWadpc&t=...
...,...,...,...,...,...,...
1116,oh man oh my god,577,579,oh man oh my god,gi6SYQyVIf8,https://www.youtube.com/watch?v=gi6SYQyVIf8&t=...
1117,oh no oh my god,106,108,oh no oh my god,Vnx9ZJS5Oes,https://www.youtube.com/watch?v=Vnx9ZJS5Oes&t=...
1118,oh no oh my god,721,724,oh no oh my god ohhhh,Poe76zLndr8,https://www.youtube.com/watch?v=Poe76zLndr8&t=...
1119,oh d oh how did,91,95,oh d oh how did,mVhs_hOfx2I,https://www.youtube.com/watch?v=mVhs_hOfx2I&t=91s


In [118]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [119]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [120]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [121]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['English_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv',
 'English_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx']

In [122]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [123]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass